<a href="https://colab.research.google.com/github/ifeLight/ml-bot/blob/main/binance-multi-timeframe-grade-review_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas-ta
!pip install backtrader[plotting]
!pip install plotly
!pip install --upgrade firebase-admin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=02a67964035498e0390dac2955a7b2c9c83aee4fbfdb112c5242a12de51ab5e4
  Stored in directory: /root/.cache/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas-ta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 5.4 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import datetime
import requests
import json
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import backtrader as bt
import pandas_ta as ta
from imblearn.over_sampling import SMOTE
import subprocess
import firebase_admin
from firebase_admin import firestore
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
from google.colab import auth
import google.auth
import itertools
import hashlib

In [3]:
auth.authenticate_user()
#Configure Google cloud project
project_id = 'ifelight'
!gcloud config set project {project_id}

Updated property [core/project].


In [4]:
# Working GCP Bucket name
bucket_name = 'ife-storage'
# Working on Firestore name
firestore_collection_name = 'trade-models'

In [5]:
binance_base_url = 'https://52on3577u3.execute-api.eu-central-1.amazonaws.com'

def get_binance_candles(symbol: str, interval='1h', limit: int = 50, **kwargs):
    url = f'{binance_base_url}/api/v3/uiKlines?symbol={symbol}&interval={interval}&limit={limit}'
    for key, value in kwargs.items():
        url += f'&{key}={value}'
    response = requests.get(url)
    result = json.loads(response.text)
    # print(result)
    def map_result(x):
        return {
            'Date': x[0],
            'Open': x[1],
            'High': x[2],
            'Low': x[3],
            'Close': x[4],
            'Volume': x[5],
        }
    mappeded_result = []
    for x in result:
        mappeded_result.append(map_result(x))
    return mappeded_result


def candles_to_df(data):
    df =  pd.DataFrame(data)
    df['Date'] = pd.to_datetime(df['Date'], unit='ms')
    df['Open'] = df['Open'].astype(float)
    df['High'] = df['High'].astype(float)
    df['Low'] = df['Low'].astype(float)
    df['Close'] = df['Close'].astype(float)
    df['Volume'] = df['Volume'].astype(float)
    df.set_index('Date', inplace=True)
    return df

def get_all_binance_candles(symbol: str, interval='1h', start_date=None, end_date=None, limit=1000):
  try:
    return load_candles_from_cloud_storage(symbol, interval, start_date, end_date)
  except FileNotFoundError:
    pass
  result = []
  raw_start_date = start_date
  raw_end_date = end_date
  start_date = pd.to_datetime(start_date) if start_date else pd.to_datetime('2015-01-01')
  end_date = pd.to_datetime(end_date) if end_date else pd.to_datetime('today')
  while True:
    candles = get_binance_candles(symbol, interval, limit, startTime=int(start_date.timestamp() * 1000), endTime=int(end_date.timestamp() * 1000))
    if len(candles) <= 1:
      break;
    result += candles
    start_date = pd.to_datetime(datetime.datetime.fromtimestamp(candles[-1]['Date'] / 1000))
  candles_df = candles_to_df(result)
  save_candles_to_cloud_storage(candles_df, symbol, interval, raw_start_date, raw_end_date)
  return candles_df

def candles_storage_file_name(symbol: str, interval='1h', start_date=None, end_date=None):
  file_name = f'binance_{symbol}_{interval}_{start_date}_{end_date}.csv'
  return file_name

def load_candles_from_cloud_storage(symbol: str, interval: str, start_date=None, end_date=None):
  file_name = candles_storage_file_name(symbol, interval, start_date, end_date)
  try:
    # Download the file from cloud storage.
    subprocess.run(['gsutil', 'cp', f'gs://{bucket_name}/trade/candles/{file_name}', f'/tmp/{file_name}'], check=True)

    # Load the data into a Pandas DataFrame.
    with open(f'/tmp/{file_name}', 'r') as f:
      return pd.read_csv(f, index_col=0, parse_dates=True)
  except subprocess.CalledProcessError:
    # Raise a FileNotFoundError if the file is not found in cloud storage.
    raise FileNotFoundError(f"File not found: gs://{bucket_name}/trade/candles/{file_name}")

def save_candles_to_cloud_storage(df: pd.DataFrame, symbol: str, interval: str, start_date, end_date):
  file_name = candles_storage_file_name(symbol, interval, start_date, end_date)
  df.to_csv(f'/tmp/{file_name}')
  !gsutil cp /tmp/{file_name} gs://{bucket_name}/trade/candles/{file_name}


In [6]:
def add_scaled_rsi(df, window=14, prefix = '', features_columns=[]):
  series = ta.rsi(df['Close'], length=window)
  column_name = f"{prefix}RSI_{window}"
  df[column_name] = series / 100
  if(column_name not in features_columns):
    features_columns.append(column_name)
  return df

def add_scaled_ema(df, window=50, prefix = '', features_columns=[]):
  series = ta.ema(df['Close'], length=window)
  column_name = f"{prefix}EMA_{window}"
  df[column_name] = series / df['Close']
  if column_name not in features_columns:
    features_columns.append(column_name)
  return df

def add_scaled_sma(df, window=50, prefix = '', features_columns=[]):
  series = ta.sma(df['Close'], length=window)
  column_name = f"{prefix}SMA_{window}"
  df[column_name] = series / df['Close']
  if column_name not in features_columns:
    features_columns.append(column_name)
  return df

def add_scaled_macd(df, prefix= '', features_columns=[], fast = 12, slow = 26, signal=9):
  macd_df = ta.macd(df['Close'], fast=fast, slow=slow, signal=signal)
  suffix = f"{fast}_{slow}_{signal}"
  macd_column_name = f"{prefix}MACD_{suffix}"
  macds_column_name = f"{prefix}MACDs_{suffix}"
  macdh_column_name = f"{prefix}MACDh_{suffix}"
  df[macd_column_name] = macd_df[macd_df.columns[0]] / df['Close']
  df[macds_column_name] = macd_df[macd_df.columns[2]] / df['Close']
  df[macdh_column_name] = macd_df[macd_df.columns[1]] / df['Close']
  if macd_column_name not in features_columns:
    features_columns.append(macd_column_name)
  if macds_column_name not in features_columns:
    features_columns.append(macds_column_name)
  if macdh_column_name not in features_columns:
    features_columns.append(macdh_column_name)
  return df

def add_scaled_bbands(df, window=20, std=2.0, prefix= '', features_columns=[]):
  bbands_df = ta.bbands(df['Close'], length=window, std=std)
  suffix = f"{window}_{std}"
  bbl_column_name = f"{prefix}BBL_{suffix}"
  bbm_column_name = f"{prefix}BBM_{suffix}"
  bbu_column_name = f"{prefix}BBU_{suffix}"
  bbb_column_name = f"{prefix}BBB_{suffix}"
  bbp_column_name = f"{prefix}BBP_{suffix}"
  df[bbl_column_name] = bbands_df[bbands_df.columns[0]] / df['Close']
  df[bbm_column_name] = bbands_df[bbands_df.columns[1]] / df['Close']
  df[bbu_column_name] = bbands_df[bbands_df.columns[2]] / df['Close']
  df[bbb_column_name] = bbands_df[bbands_df.columns[3]]
  df[bbp_column_name] = bbands_df[bbands_df.columns[4]]
  if bbl_column_name not in features_columns: features_columns.append(bbl_column_name)
  if bbm_column_name not in features_columns: features_columns.append(bbm_column_name)
  if bbu_column_name not in features_columns: features_columns.append(bbu_column_name)
  if bbb_column_name not in features_columns: features_columns.append(bbb_column_name)
  if bbp_column_name not in features_columns: features_columns.append(bbp_column_name)
  return df

In [7]:
def merge_candlesticks_data(df1, df2):
  """
  Merges two candlestick DataFrames with forward fill, handling different timeframes,
  and prevents duplicate OHLCV columns.
  Ensure both DataFrames have a datetime index.
  And ensure the second DataFrame is the larger timeframe

  Args:
      df1: First candlestick DataFrame with datetime index.
      df2: Second candlestick DataFrame with datetime index.

  Returns:
      Merged DataFrame with forward-filled values, and no duplicate OHLCV columns.
  """
  # Ensure both DataFrames have a datetime index
  if not isinstance(df1.index, pd.DatetimeIndex) or not isinstance(df2.index, pd.DatetimeIndex):
      raise ValueError("DataFrames must have a datetime index.")
  # Identify OHLCV columns
  ohlcv_cols = ['open', 'high', 'low', 'close', 'volume']

  # Rename columns in df2 that conflict with df1's OHLCV columns
  for col in df2.columns:
    if col.lower() in ohlcv_cols and col.lower() in df1.columns.str.lower():
      del df2[col]

  # Merge the DataFrames using outer join, which preserves all dates
  merged_df = pd.merge(df1, df2, how='outer', left_index=True, right_index=True, suffixes=('_df1', '_df2'))

  # Forward fill the missing values for each column
  for col in merged_df.columns:
    merged_df[col] = merged_df[col].ffill()

  return merged_df

In [8]:
def resample_candles(df, interval='1h'):
    return df.resample(interval).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}).ffill()

In [9]:
def predict_in_batches(model, test_data, features_columns, seq_length, batch_size=128):
    """
    Predicts in batches to speed up inference.
    """
    num_samples = len(test_data) - seq_length
    feature_array = test_data[features_columns].values
    predicted_values = np.full(len(test_data), np.nan)

    for start_idx in range(0, num_samples, batch_size):
        end_idx = min(start_idx + batch_size, num_samples)
        batch_indices = range(start_idx + seq_length, end_idx + seq_length)
        batch_input = np.array([feature_array[i - seq_length:i] for i in batch_indices])

        if len(batch_input) > 0 :
            predictions = model.predict(batch_input, verbose=0)
            predicted_values[batch_indices] = predictions.flatten() #Flatten to 1d array.

    test_data['Predicted_Value'] = predicted_values
    return test_data

In [10]:
class PredictedValueStrategy(bt.Strategy):
    params = (
        ('buy_threshold', 0.6),
        ('sell_threshold', 0.6),
        ('leverage', 1),  # Leverage ratio
        ('margin', 1000),
        ('log', True)
    )

    def __init__(self):
        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add the Predicted_Value as a data feed
        self.predicted_value = self.datas[0].predicted_value

    def next(self):
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
         # # Check if we are in the market
        if not self.position:
          cash = self.broker.getcash()
          position_size = (self.params.margin * self.params.leverage) / self.data.close[0]
          # Long signal
          if self.predicted_value[0] > self.params.buy_threshold:
            self.log('LONG POSITION CREATED, %.2f' % self.datas[0].close[0])
            self.order = self.buy(size=position_size)

          # Short signal
          elif self.predicted_value[0] < self.params.sell_threshold:
            self.log('SHORT POSITION CREATED, %.2f' % self.datas[0].close[0])
            self.order = self.sell(size=position_size)

        else:
          if self.predicted_value[0] > self.params.buy_threshold and self.position.size < 0:
            self.log('CLOSE SHORT POSITION CREATED, %.2f' % self.datas[0].close[0])
            self.order = self.close()
          elif self.predicted_value[0] < self.params.sell_threshold and self.position.size > 0:
            self.log('CLOSE LONG POSITION CREATED, %.2f' % self.datas[0].close[0])
            self.order = self.close()

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return
        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            if order.status == order.Canceled:
                self.log('Order Canceled')
            elif order.status == order.Margin:
                self.log(f'Order Margin Not Enough - Available cash: {self.broker.getcash()}')
            elif order.status == order.Rejected:
                self.log('Order Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.log(f'TRADE COMPLETED, GROSS {trade.pnl:.2f}, NET {trade.pnlcomm:.2f}, Available Cash {self.broker.getcash():.2f}')

    def log(self, txt, dt=None):
        if not self.params.log:
          return
        dt = dt or self.datas[0].datetime.date(0)
        time = self.datas[0].datetime.time()
        print(f'{dt.isoformat()} {time.isoformat()}, {txt}')


In [11]:
# Extend PandasData to include the custom column
class CustomPandasData(bt.feeds.PandasData):
    # Add custom columns
    lines = ('predicted_value',)  # Add the custom line
    params = (
        ('predicted_value', 'Predicted_Value'),  # Map the column name
    )


In [12]:
def get_model_cloud_storage_path(bucket_name, local_file_name):
    """
    Returns the cloud storage path for a given model name.

    Parameters:
        local_file_name (str): The name of the model.

    Returns:
        str: The cloud storage path. (e.g., gs://<bucket_name>/trade/models/<model_name>.h5).
    """
    return f'gs://{bucket_name}/trade/models/{local_file_name}'

def save_model_to_cloud_storage(model: tf.keras.Model, model_name: str, bucket_name: str):
    """
    Saves a TensorFlow model to Google Cloud Storage and returns the cloud storage file path.

    Parameters:
        model (tf.keras.Model): The TensorFlow model to save.
        model_name (str): The name of the model (used to create the file name).
        bucket_name (str): The name of the Google Cloud Storage bucket.

    Returns:
        str: The cloud storage file path (e.g., gs://<bucket_name>/trade/models/<model_name>.h5).
    """
    # Define the local and cloud storage file paths
    local_file_name = f'{model_name}.h5'
    local_file_path = f'/tmp/{local_file_name}'
    cloud_file_path = get_model_cloud_storage_path(bucket_name, local_file_name)

    # Save the model locally
    model.save(local_file_path)

    try:
        # Upload the model to Google Cloud Storage
        subprocess.run(['gsutil', 'cp', local_file_path, cloud_file_path], check=True)
        print(f"Model saved to {cloud_file_path}")
    except subprocess.CalledProcessError as e:
        # Handle errors during the upload process
        raise RuntimeError(f"Failed to upload model to Google Cloud Storage: {e}")
    finally:
        # Clean up the local file
        if os.path.exists(local_file_path):
            os.remove(local_file_path)
    return cloud_file_path

def load_model_from_cloud_storage(model_name: str, bucket_name: str):
    """
    Loads a TensorFlow model from Google Cloud Storage.

    Parameters:
        model_name (str): The name of the model (used to create the file name).
        bucket_name (str): The name of the Google Cloud Storage bucket.

    Returns:
        tf.keras.Model: The loaded TensorFlow model.
    """
    # Define the local and cloud storage file paths
    local_file_name = f'{model_name}.h5'
    local_file_path = f'/tmp/{local_file_name}'
    cloud_file_path = get_model_cloud_storage_path(bucket_name, local_file_name)

    try:
        # Download the model from Google Cloud Storage
        subprocess.run(['gsutil', 'cp', cloud_file_path, local_file_path], check=True)

        # Load the model
        model = tf.keras.models.load_model(local_file_path)
        print(f"Model loaded from {cloud_file_path}")
        return model
    except subprocess.CalledProcessError as e:
        raise FileNotFoundError(f"Model not found in Google Cloud Storage: {e}")
    finally:
        # Clean up the local file
        if os.path.exists(local_file_path):
            os.remove(local_file_path)


In [13]:
def initialize_firestore(project_id):
    """
    Initialize the Firestore client using the service account key.

    Parameters:
        project_id (str): The Google Cloud project ID.

    Returns:
        firestore.Client: Initialized Firestore client.
    """
    if not firebase_admin._apps:
        # cred = credentials.Certificate(service_account_key_path)
        cred = firebase_admin.credentials.ApplicationDefault()
        cred._project_id = project_id  # Add this line
        os.environ["GOOGLE_CLOUD_PROJECT"] = project_id
        firebase_admin.initialize_app(cred, {'projectId': project_id})
    return firestore.client()


def save_to_firestore(project_id, collection_name, data):
    """
    Save data to a Firestore collection.

    Parameters:
        project_id (str): The Google Cloud project ID.
        service_account_key_path (str): Path to the Firebase service account key JSON file.
        collection_name (str): Name of the Firestore collection.
        data (dict): Data to save in the document.

    Returns:
        None
    """
    # Initialize Firestore client
    db = initialize_firestore(project_id)

    # Save data to Firestore
    doc_ref = db.collection(collection_name).document()
    doc_ref.set(data)
    print(f"Data saved to Firestore: Collection={collection_name}, Document ID={doc_ref.id}")

In [49]:
def get_nested_value(data, keys):
    """
    Retrieves a deeply nested value from a dictionary or list.

    Args:
        data: The dictionary or list to traverse.
        keys: A list of keys or indices to access the nested value.

    Returns:
        The nested value if found, otherwise None.
    """
    try:
        current_value = data
        for key in keys:
            if isinstance(current_value, dict):
                current_value = current_value.get(key)
            elif isinstance(current_value, list):
                if isinstance(key, int) and 0 <= key < len(current_value):
                    current_value = current_value[key]
                else:
                    return None #invalid index
            else:
                return None #not a dict or list
            if current_value is None:
                return None  # Early exit if any key is missing
        return current_value
    except (TypeError, IndexError, AttributeError, KeyError):
        return None

In [14]:
trade_models = []
db = initialize_firestore(project_id)
trade_moldel_ref = db.collection('trade-models')
trade_models_stream = (
    trade_moldel_ref.where('version', '==', 'v1')
    .where('symbol', '==', 'BTCUSDT')
    .where('exchange', '==', 'binance')
    .where('timeframe', '==', '15m')
    .stream()
)
for model in trade_models_stream:
    trade_models.append(model.to_dict())
trade_models

/usr/local/lib/python3.11/dist-packages/google/cloud/firestore_v1/base_collection.py:303: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
<ipython-input-14-039c559c4f85>:6: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  .where('symbol', '==', 'BTCUSDT')
<ipython-input-14-039c559c4f85>:7: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  .where('exchange', '==', 'binance')
<ipython-input-14-039c559c4f85>:8: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  .where('timeframe', '==', '15m')


[{'model_path': 'gs://ife-storage/trade/models/binance_mtf_v1_15m_01-01-2021_01-01-2024_ca605b32.h5',
  'val_mae': 0.42670920491218567,
  'train_candles_length': 66944,
  'trade_buy_threshold': 0.5,
  'features_columns': ['15m_RSI_14',
   '15m_RSI_6',
   '15m_EMA_5',
   '15m_EMA_21',
   '15m_SMA_50',
   '15m_MACD_12_26_9',
   '15m_MACDs_12_26_9',
   '15m_MACDh_12_26_9',
   '15m_BBL_20_2.0',
   '15m_BBM_20_2.0',
   '15m_BBU_20_2.0',
   '15m_BBB_20_2.0',
   '15m_BBP_20_2.0',
   '1h_RSI_14',
   '1h_RSI_6',
   '1h_EMA_5',
   '1h_EMA_21',
   '1h_MACD_12_26_9',
   '1h_MACDs_12_26_9',
   '1h_MACDh_12_26_9',
   '1h_BBL_20_2.0',
   '1h_BBM_20_2.0',
   '1h_BBU_20_2.0',
   '1h_BBB_20_2.0',
   '1h_BBP_20_2.0',
   '4h_RSI_14',
   '4h_RSI_6',
   '4h_EMA_5',
   '4h_EMA_21'],
  'training_data_ratio': 0.8,
  'sharpe_analysis': {'sharperatio': None},
  'model_input_shape': [100, 29],
  'epochs': 20,
  'training_data_end_date': DatetimeWithNanoseconds(2023, 5, 27, 14, 0, tzinfo=datetime.timezone.utc),
  

In [15]:
data = pd.DataFrame(trade_models)
data

,model_path,val_mae,train_candles_length,trade_buy_threshold,features_columns,training_data_ratio,sharpe_analysis,model_input_shape,epochs,training_data_end_date,...,exchange,starting_portfolio_value,symbol,test_data_start_date,middle_timeframe,trade_sell_threshold,train_loss,trade_leverage,final_portfolio_value,start_date
0,gs://ife-storage/trade/models/binance_mtf_v1_1...,0.426709,66944,0.5,"[15m_RSI_14, 15m_RSI_6, 15m_EMA_5, 15m_EMA_21,...",0.8,{'sharperatio': None},"[100, 29]",20,2023-05-27 14:00:00+00:00,...,binance,10000.0,BTCUSDT,2023-05-28 15:15:00+00:00,1h,-0.5,0.228014,20,48801.284621,01-01-2021
1,gs://ife-storage/trade/models/binance_mtf_v1_1...,0.421798,48737,0.8,"[15m_RSI_14, 15m_RSI_6, 15m_EMA_5, 15m_EMA_21,...",0.8,{'sharperatio': 0},"[100, 29]",20,2024-10-01 00:15:00+00:00,...,binance,10000.0,BTCUSDT,2024-10-02 01:30:00+00:00,1h,-0.5,0.244409,15,4369.235279,01-01-2023
2,gs://ife-storage/trade/models/binance_mtf_v1_1...,0.399768,66980,0.8,"[15m_RSI_14, 15m_RSI_6, 15m_EMA_5, 15m_EMA_21,...",0.8,{'sharperatio': None},"[100, 29]",20,2024-05-26 13:00:00+00:00,...,binance,10000.0,BTCUSDT,2024-05-27 14:15:00+00:00,1h,-0.7,0.234009,10,16087.619855,01-01-2022
3,gs://ife-storage/trade/models/binance_mtf_v1_1...,0.425303,66767,0.5,"[15m_RSI_14, 15m_RSI_6, 15m_EMA_5, 15m_EMA_21,...",0.8,{'sharperatio': None},"[100, 29]",20,2020-05-26 23:15:00+00:00,...,binance,10000.0,BTCUSDT,2020-05-28 00:30:00+00:00,1h,-0.6,0.241986,7,36824.672361,01-01-2018
4,gs://ife-storage/trade/models/binance_mtf_v1_1...,0.399768,66980,0.6,"[15m_RSI_14, 15m_RSI_6, 15m_EMA_5, 15m_EMA_21,...",0.8,{'sharperatio': None},"[100, 29]",20,2024-05-26 13:00:00+00:00,...,binance,10000.0,BTCUSDT,2024-05-27 14:15:00+00:00,1h,-0.7,0.234009,15,50920.109695,01-01-2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,gs://ife-storage/trade/models/binance_mtf_v1_1...,0.421798,48737,0.6,"[15m_RSI_14, 15m_RSI_6, 15m_EMA_5, 15m_EMA_21,...",0.8,{'sharperatio': 14},"[100, 29]",20,2024-10-01 00:15:00+00:00,...,binance,10000.0,BTCUSDT,2024-10-02 01:30:00+00:00,1h,-0.5,0.244409,15,39518.435190,01-01-2023
500,gs://ife-storage/trade/models/binance_mtf_v1_1...,0.387304,66855,0.7,"[15m_RSI_14, 15m_RSI_6, 15m_EMA_5, 15m_EMA_21,...",0.8,{'sharperatio': None},"[100, 29]",20,2021-05-26 17:15:00+00:00,...,binance,10000.0,BTCUSDT,2021-05-27 18:30:00+00:00,1h,-0.6,0.263646,25,34754.316531,01-01-2019
501,gs://ife-storage/trade/models/binance_mtf_v1_1...,0.421798,48737,0.7,"[15m_RSI_14, 15m_RSI_6, 15m_EMA_5, 15m_EMA_21,...",0.8,{'sharperatio': 2},"[100, 29]",20,2024-10-01 00:15:00+00:00,...,binance,10000.0,BTCUSDT,2024-10-02 01:30:00+00:00,1h,-0.5,0.244409,25,20733.499548,01-01-2023
502,gs://ife-storage/trade/models/binance_mtf_v1_1...,0.404841,66915,0.6,"[15m_RSI_14, 15m_RSI_6, 15m_EMA_5, 15m_EMA_21,...",0.8,{'sharperatio': None},"[100, 29]",20,2022-05-26 19:00:00+00:00,...,binance,10000.0,BTCUSDT,2022-05-27 20:15:00+00:00,1h,-0.5,0.242630,10,57054.178927,01-01-2020


In [16]:
sorted(data.columns)

['broker_commision',
 'created_at',
 'drawdown_analysis',
 'end_date',
 'epochs',
 'exchange',
 'features_columns',
 'final_epoch',
 'final_portfolio_value',
 'higher_timeframe',
 'lower_timeframe',
 'middle_timeframe',
 'model_input_shape',
 'model_name',
 'model_path',
 'seq_length',
 'sharpe_analysis',
 'start_date',
 'starting_portfolio_value',
 'symbol',
 'test_candles_length',
 'test_data_end_date',
 'test_data_start_date',
 'timeframe',
 'trade_analysis',
 'trade_buy_threshold',
 'trade_leverage',
 'trade_margin',
 'trade_sell_threshold',
 'train_candles_length',
 'train_loss',
 'train_mae',
 'training_data_end_date',
 'training_data_ratio',
 'training_data_start_date',
 'val_candles_length',
 'val_loss',
 'val_mae',
 'version']

In [20]:
performing_model = data[['start_date', 'end_date', 'final_portfolio_value', 'trade_buy_threshold', 'trade_sell_threshold', 'trade_leverage', 'val_loss', 'val_mae', 'test_data_start_date', 'test_data_end_date']].sort_values(by='final_portfolio_value', ascending=False).head(20)
performing_model

,start_date,end_date,final_portfolio_value,trade_buy_threshold,trade_sell_threshold,trade_leverage,val_loss,val_mae,test_data_start_date,test_data_end_date
334,01-01-2019,31-12-2021,136371.757700,0.5,-0.5,25,0.224315,0.387304,2021-05-27 18:30:00+00:00,2021-12-30 19:30:00+00:00
124,01-01-2020,31-12-2022,124316.117028,0.5,-0.5,25,0.242672,0.404841,2022-05-27 20:15:00+00:00,2022-12-30 21:00:00+00:00
362,01-01-2020,31-12-2022,122611.538572,0.6,-0.5,25,0.242672,0.404841,2022-05-27 20:15:00+00:00,2022-12-30 21:00:00+00:00
343,01-01-2019,31-12-2021,114265.077466,0.5,-0.5,20,0.224315,0.387304,2021-05-27 18:30:00+00:00,2021-12-30 19:30:00+00:00
331,01-01-2019,31-12-2021,109128.855969,0.5,-0.6,25,0.224315,0.387304,2021-05-27 18:30:00+00:00,2021-12-30 19:30:00+00:00
412,01-01-2022,31-12-2024,107671.489501,0.5,-0.5,25,0.241482,0.399768,2024-05-27 14:15:00+00:00,2024-12-30 20:15:00+00:00
417,01-01-2020,31-12-2022,104617.752425,0.5,-0.5,20,0.242672,0.404841,2022-05-27 20:15:00+00:00,2022-12-30 21:00:00+00:00
374,01-01-2020,31-12-2022,101699.383001,0.6,-0.5,20,0.242672,0.404841,2022-05-27 20:15:00+00:00,2022-12-30 21:00:00+00:00
198,01-01-2020,31-12-2022,99460.295528,0.7,-0.5,25,0.242672,0.404841,2022-05-27 20:15:00+00:00,2022-12-30 21:00:00+00:00
218,01-01-2022,31-12-2024,98868.281067,0.5,-0.6,25,0.241482,0.399768,2024-05-27 14:15:00+00:00,2024-12-30 20:15:00+00:00


In [63]:
data.loc[334]['trade_analysis']['won']

{'total': 334, 'pnl': {'average': 506, 'total': 168564, 'max': 2852}}

In [50]:
get_nested_value(data.loc[334]['trade_analysis'], ['streak', 'won', 'longest'])

20

In [64]:
def get_trade_analysis_overview(df):
  selected_columns = [
      'start_date', 'end_date', 'final_portfolio_value', 'trade_buy_threshold',
      'trade_sell_threshold', 'trade_leverage', 'val_loss', 'val_mae',
      'test_data_start_date', 'test_data_end_date', 'trade_analysis', 'drawdown_analysis'
  ]
  df = df[selected_columns].copy().sort_values(by='final_portfolio_value', ascending=False)
  df['total_trade'] = df['trade_analysis'].apply(lambda x: x['total']['total'])
  df['longest_won_streak'] = df['trade_analysis'].apply(lambda x: get_nested_value(x, ['streak', 'won', 'longest']))
  df['longest_lost_streak'] = df['trade_analysis'].apply(lambda x: get_nested_value(x, ['streak', 'lost', 'longest']))
  df['total_pnl'] = df['trade_analysis'].apply(lambda x: get_nested_value(x, ['pnl', 'net', 'total']))
  df['avg_pnl'] = df['trade_analysis'].apply(lambda x: get_nested_value(x, ['pnl', 'net', 'average']))
  df['total_win_trade'] = df['trade_analysis'].apply(lambda x: get_nested_value(x, ['won', 'total']))
  df['total_won_amount'] = df['trade_analysis'].apply(lambda x: get_nested_value(x, ['won', 'pnl', 'total']))
  df['max_won_amount'] = df['trade_analysis'].apply(lambda x: get_nested_value(x, ['won', 'pnl', 'max']))
  df['avg_won_amount'] = df['trade_analysis'].apply(lambda x: get_nested_value(x, ['won', 'pnl', 'average']))
  df['total_loss_trade'] = df['trade_analysis'].apply(lambda x: get_nested_value(x, ['lost', 'total']))
  df['total_lost_amount'] = df['trade_analysis'].apply(lambda x: get_nested_value(x, ['lost', 'pnl', 'total']))
  df['max_lost_amount'] = df['trade_analysis'].apply(lambda x: get_nested_value(x, ['lost', 'pnl', 'max']))
  df['avg_lost_amount'] = df['trade_analysis'].apply(lambda x: get_nested_value(x, ['lost', 'pnl', 'average']))
  df['max_drawdown_percent'] = df['drawdown_analysis'].apply(lambda x: get_nested_value(x, ['max', 'drawdown']))
  df['max_drawdown_value'] = df['drawdown_analysis'].apply(lambda x: get_nested_value(x, ['max', 'moneydown']))
  df['max_drawdown_len'] = df['drawdown_analysis'].apply(lambda x: get_nested_value(x, ['max', 'len']))
  del df['trade_analysis']
  del df['drawdown_analysis']
  return df

In [65]:
trade_data = get_trade_analysis_overview(data)
trade_data

,start_date,end_date,final_portfolio_value,trade_buy_threshold,trade_sell_threshold,trade_leverage,val_loss,val_mae,test_data_start_date,test_data_end_date,...,total_won_amount,max_won_amount,avg_won_amount,total_loss_trade,total_lost_amount,max_lost_amount,avg_lost_amount,max_drawdown_percent,max_drawdown_value,max_drawdown_len
334,01-01-2019,31-12-2021,136371.757700,0.5,-0.5,25,0.224315,0.387304,2021-05-27 18:30:00+00:00,2021-12-30 19:30:00+00:00,...,168564.0,2852.0,506.0,68.0,-42032.0,-2864.0,-618.0,20,4610,852
124,01-01-2020,31-12-2022,124316.117028,0.5,-0.5,25,0.242672,0.404841,2022-05-27 20:15:00+00:00,2022-12-30 21:00:00+00:00,...,127752.0,2970.0,494.0,42.0,-13600.0,-2432.0,-324.0,10,4062,790
362,01-01-2020,31-12-2022,122611.538572,0.6,-0.5,25,0.242672,0.404841,2022-05-27 20:15:00+00:00,2022-12-30 21:00:00+00:00,...,124380.0,4108.0,558.0,32.0,-11932.0,-2152.0,-372.0,8,3306,604
343,01-01-2019,31-12-2021,114265.077466,0.5,-0.5,20,0.224315,0.387304,2021-05-27 18:30:00+00:00,2021-12-30 19:30:00+00:00,...,140056.0,2282.0,408.0,72.0,-35662.0,-2292.0,-494.0,18,3688,770
331,01-01-2019,31-12-2021,109128.855969,0.5,-0.6,25,0.224315,0.387304,2021-05-27 18:30:00+00:00,2021-12-30 19:30:00+00:00,...,129570.0,2656.0,632.0,52.0,-30278.0,-2694.0,-582.0,22,3856,788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,01-01-2018,31-12-2020,-14389.286926,0.8,-0.6,20,0.266579,0.425303,2020-05-28 00:30:00+00:00,2020-12-30 20:45:00+00:00,...,2744.0,1946.0,550.0,4.0,-10142.0,-4172.0,-2536.0,226,25892,17832
232,01-01-2018,31-12-2020,-15364.986535,0.8,-0.5,20,0.266579,0.425303,2020-05-28 00:30:00+00:00,2020-12-30 20:45:00+00:00,...,2608.0,2368.0,1304.0,8.0,-11090.0,-4172.0,-1584.0,242,26236,17832
268,01-01-2018,31-12-2020,-19401.430975,0.8,-0.7,25,0.266579,0.425303,2020-05-28 00:30:00+00:00,2020-12-30 20:45:00+00:00,...,3162.0,1596.0,790.0,4.0,-12716.0,-5176.0,-3178.0,252,32382,17832
498,01-01-2018,31-12-2020,-20486.608657,0.8,-0.6,25,0.266579,0.425303,2020-05-28 00:30:00+00:00,2020-12-30 20:45:00+00:00,...,3432.0,2432.0,686.0,4.0,-12678.0,-5214.0,-3170.0,274,32364,17832


In [59]:
trade_data.columns

Index(['start_date', 'end_date', 'final_portfolio_value',
       'trade_buy_threshold', 'trade_sell_threshold', 'trade_leverage',
       'val_loss', 'val_mae', 'test_data_start_date', 'test_data_end_date',
       'total_trade', 'longest_won_streak', 'longest_lost_streak', 'total_pnl',
       'avg_pnl', 'total_win_trade', 'total_won_amount', 'max_won_amount',
       'avg_won_amount', 'total_loss_trade', 'total_lost_amount',
       'max_lost_amount', 'avg_lost_amount', 'max_drawdown_percent',
       'max_drawdown_value', 'max_drawdown_len'],
      dtype='object')

In [67]:
trade_data.query('max_drawdown_percent < 20 and final_portfolio_value > 50000')

,start_date,end_date,final_portfolio_value,trade_buy_threshold,trade_sell_threshold,trade_leverage,val_loss,val_mae,test_data_start_date,test_data_end_date,...,total_won_amount,max_won_amount,avg_won_amount,total_loss_trade,total_lost_amount,max_lost_amount,avg_lost_amount,max_drawdown_percent,max_drawdown_value,max_drawdown_len
124,01-01-2020,31-12-2022,124316.117028,0.5,-0.5,25,0.242672,0.404841,2022-05-27 20:15:00+00:00,2022-12-30 21:00:00+00:00,...,127752.0,2970.0,494.0,42.0,-13600.0,-2432.0,-324.0,10,4062,790
362,01-01-2020,31-12-2022,122611.538572,0.6,-0.5,25,0.242672,0.404841,2022-05-27 20:15:00+00:00,2022-12-30 21:00:00+00:00,...,124380.0,4108.0,558.0,32.0,-11932.0,-2152.0,-372.0,8,3306,604
343,01-01-2019,31-12-2021,114265.077466,0.5,-0.5,20,0.224315,0.387304,2021-05-27 18:30:00+00:00,2021-12-30 19:30:00+00:00,...,140056.0,2282.0,408.0,72.0,-35662.0,-2292.0,-494.0,18,3688,770
412,01-01-2022,31-12-2024,107671.489501,0.5,-0.5,25,0.241482,0.399768,2024-05-27 14:15:00+00:00,2024-12-30 20:15:00+00:00,...,117414.0,1722.0,320.0,82.0,-19742.0,-976.0,-244.0,12,2398,848
417,01-01-2020,31-12-2022,104617.752425,0.5,-0.5,20,0.242672,0.404841,2022-05-27 20:15:00+00:00,2022-12-30 21:00:00+00:00,...,105306.0,2376.0,398.0,44.0,-10820.0,-1946.0,-252.0,8,3250,790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,01-01-2018,31-12-2020,51419.982532,0.5,-0.5,10,0.266579,0.425303,2020-05-28 00:30:00+00:00,2020-12-30 20:45:00+00:00,...,52358.0,800.0,140.0,76.0,-10200.0,-754.0,-134.0,6,1162,796
328,01-01-2017,01-01-2020,50971.335708,0.5,-0.5,20,0.259467,0.423599,2019-07-13 10:00:00+00:00,2019-12-31 18:00:00+00:00,...,58370.0,1696.0,394.0,40.0,-17682.0,-3006.0,-452.0,18,4620,2706
4,01-01-2022,31-12-2024,50920.109695,0.6,-0.7,15,0.241482,0.399768,2024-05-27 14:15:00+00:00,2024-12-30 20:15:00+00:00,...,45260.0,3324.0,408.0,18.0,-4808.0,-918.0,-266.0,12,2466,974
43,01-01-2023,31-12-2025,50749.959564,0.5,-0.5,15,0.252373,0.421798,2024-10-02 01:30:00+00:00,2025-03-08 22:15:00+00:00,...,44312.0,1682.0,372.0,20.0,-3350.0,-748.0,-176.0,8,1474,580
